In [2]:
import ROOT #Start by importing ROOT
import numpy as np
ROOT.enableJSVis() #need this to make interactive ROOT plots
import os
import time
c1 = ROOT.TCanvas() #declare the ROOT canvas
#ROOT.gSystem.Load("/home/vasu/quadis/build/lib/libquadis.so")
ROOT.gSystem.Load("/usr/local/software/quadis/build/lib/libquadis.so")   

Welcome to JupyROOT 6.10/00


0

In [3]:
#Grab a pulse from the data
infile = ROOT.TFile("/home/vasu/PPC/PPC_SigGen/CleanNoisyPulses/rz200.root")
# infile = ROOT.TFile("/home/vasu/PPC/PPC_SigGen/June_2018/RUN10000000_0.root")
tree = infile.Get("T1")
nevent = ROOT.NEvent()
tree.SetBranchAddress("NEvent",nevent)
tree.GetEntry(1)
npulse = nevent.GetPulse(0) #the original pulse
# npulse = normalize(npulse)

#Initialize the fitter
fitter = ROOT.NAPulseFitter("/home/vasu/PPC/PPC_SigGen/SignalLib_root/new_OutputRZperfect.root","T1","NEvent",1)

# Parameters to pre-process pulse
baselines = 1000
taus = 7100
rises=700
gaps=225
pulse = fitter.GetPreProcessedPulse(npulse, False, baselines, taus, rises, gaps)# true applies pole-zero
ph = npulse.GetTH1Dmus()
ph.Draw()
c1.Draw()

#Prepare the fitter
fitter.SetLibPulseMinThreshold(0.01)
fitter.SetLibPulseMaxThreshold(0.75)
# fitter.FixNDOF() #if true, maxthreshold ignored
# fitter.SetNDOF(100)
fitter.SetShiftWindow(5) #shift by +/- this much around point of alignment
#Do the fit:
fitter.FitPulse(pulse)
#Print the best fit
fitter.PrintBestFit()
#fitter.PrintFitResult(1466)

Loaded library tree with 1724 entries
Best fit index: 1497 with chi-square: 0.0352853 NDOF: 102 shifted: -1 start data sample: 1011 and prob: 1


In [4]:
#cfdFilter = ROOT.NFCFD(1000,0.7,1000)
#cfdPulse = ROOT.NPulse()
#cfdFilter.Filter(pulse,cfdPulse)
#print("CFD Min at ",cfdFilter.GetCalcParameterByName("CFDNS"))
#libpulse = fitter.GetLibPulse(1365)
#lh = libpulse.GetTH1Dmus()
#lh.Draw()
#c1.Draw()

libpulse = fitter.GetBestFitPulse()
shift = fitter.GetBestShift()
ph = pulse.GetTH1Dmus()
ph.Draw()

lh = libpulse.GetTH1Dmus()
lh.SetLineColor(2)
lh.SetLineWidth(3)
lh.GetXaxis().SetLimits(8*shift/1000.,8*shift/1000.+lh.GetNbinsX()*8/1000.)
lh.Draw("same")
c1.Draw()

In [5]:
print nevent.GetPulseData(0).GetParameterByName("R")
print nevent.GetPulseData(0).GetParameterByName("Z")

# print fitter.GetLibPulseData(530).GetParameterByName("R")
# print fitter.GetLibPulseData(530).GetParameterByName("Z")

31.0
27.0


In [5]:
def quicksort(x,y):
    if len(x) == 1 or len(x) == 0:
        return x,y;
    else:
        pivot = x[0]
        i = 0
        for j in range(len(x)-1):
            if x[j+1] < pivot:
                x[j+1],x[i+1] = x[i+1], x[j+1]
                y[j+1],y[i+1] = y[i+1], y[j+1]
                i += 1
        x[0],x[i] = x[i],x[0]
        y[0],y[i] = y[i],y[0]
        first1,first2 = quicksort(x[:i],y[:i])
        second1,second2 = quicksort(x[i+1:],y[i+1:])
        first1 = np.append(first1,x[i])
        first2 = np.append(first2,y[i])
        array1 = np.append(first1,second1)
        array2 = np.append(first2,second2)
        return array1,array2;

In [6]:
chis = fitter.GetChis()

# hchi = ROOT.TH1F("hchi","hchi", 1000, 0., 10)
# hr = ROOT.TH1F("hr","hr", 20, 0., 55)
# i = 0

# print chis
# for chi in chis:
#     hchi.Fill(chi)
# hchi.Draw()
# c1.Draw()
# print hchi.GetMean()
# print hchi.GetStdDev()

In [7]:
# hr.Draw()
# c1.Draw()

In [8]:
start = time.time()
minmax = ROOT.NPCMinMax()
hR = ROOT.TH1F("hR","hR", 100, 0., 100)
hZ = ROOT.TH1F("hZ","hZ", 100, 0., 100)
hRZ = ROOT.TH2F("hRZ","hRZ", 100, 0., 100, 100, 0., 100)
hallchi = ROOT.TH1F("hallchi","hcallhi", 1000, 0., 10)
hE = ROOT.TH1F("hE","hE", 200, 0., 2e3)
hEchi = ROOT.TH2F("hEchi","hEchi", 200, 0., 2e3, 1000,0.,10)
AoverEvschi = ROOT.TH2F("AoverEvschi","AoverE vs chi;chi;A/E",1000, 0., 10,100,0,0.003)

trapPulse = ROOT.NPulse() #to hold the filtered result
#Initialize the filter (using multiples of the sample period for gap, peaking, tau)
trapFilter = ROOT.NFTrapezoid(6000,1800,6750*8) #tau of about 6000 samples
trapFilter.SetRiseNS(6000)
trapFilter.SetGapNS(1800)
#trapFilter.SetBaseline(baseline)
trapFilter.Filter(npulse,trapPulse)

#Speed test... it's slow, 2000 takes about 25min
# data_ind = []
# num_files=0
# f = open('rt_data_ind', 'r')
# while True:
#     line = f.readline()
#     if not line:
#         break
#     else:
#         temp = line.strip("\n")
#         data_ind.append(int(temp))
#     num_files = num_files+1
# f.close()
for j in range (0,1000): #Take 1000 files only
#     j = data_ind[data_f]
#     print j
    tree.GetEntry(j)
    npulse = nevent.GetPulse(0)
    trapFilter.Filter(npulse,trapPulse)
    energy = trapFilter.GetCalcParameterByName("Max")
    hE.Fill(energy)
      
    pulse = fitter.GetPreProcessedPulse(npulse, True, baselines, taus, rises, gaps)
    fitter.FitPulse(pulse)
    hallchi.Fill(fitter.GetBestChiSquare())
    hEchi.Fill(energy,fitter.GetBestChiSquare())
    
    sgdPulse = ROOT.NPulse()
    sgFilter = ROOT.NFSavitzkyGolay()
    sgFilter.SetSGType(5);
    sgFilter.Filter(npulse,sgdPulse)
    minmax.CalculateParameters(sgdPulse)
    A = minmax.GetCalcParameterByName("Max")
    E = nevent.GetPulseData(0).GetParameterByNameUL("MaxEnergy")
    AoverEvschi.Fill(fitter.GetBestChiSquare(),A/E)
    
#     if fitter.GetBestChiSquare() > 1:
#         continue
    R = fitter.GetBestFitPulseData().GetParameterByName("R")
    Z = fitter.GetBestFitPulseData().GetParameterByName("Z")
    hR.Fill(R)
    hZ.Fill(Z)
    hRZ.Fill(R,Z)
    
end = time.time()
print "TIME =",(end - start)

TIME = 1152.20759702
Missing parameter is: MaxEnergy
Missing parameter is: MaxEnergy
Missing parameter is: MaxEnergy
Missing parameter is: MaxEnergy
Missing parameter is: MaxEnergy
Missing parameter is: MaxEnergy
Missing parameter is: MaxEnergy
Missing parameter is: MaxEnergy
Missing parameter is: MaxEnergy
Missing parameter is: MaxEnergy
Missing parameter is: MaxEnergy
Missing parameter is: MaxEnergy
Missing parameter is: MaxEnergy
Missing parameter is: MaxEnergy
Missing parameter is: MaxEnergy
Missing parameter is: MaxEnergy
Missing parameter is: MaxEnergy
Missing parameter is: MaxEnergy
Missing parameter is: MaxEnergy
Missing parameter is: MaxEnergy
Missing parameter is: MaxEnergy
Missing parameter is: MaxEnergy
Missing parameter is: MaxEnergy
Missing parameter is: MaxEnergy
Missing parameter is: MaxEnergy
Missing parameter is: MaxEnergy
Missing parameter is: MaxEnergy
Missing parameter is: MaxEnergy
Missing parameter is: MaxEnergy
Missing parameter is: MaxEnergy
Missing parameter i

Warning in <NParameterList<unsigned long>::GetParameterIndexByName()>: No such parameter
Warning in <NParameterList<unsigned long>::GetParameterIndexByName()>: No such parameter
Warning in <NParameterList<unsigned long>::GetParameterIndexByName()>: No such parameter
Warning in <NParameterList<unsigned long>::GetParameterIndexByName()>: No such parameter
Warning in <NParameterList<unsigned long>::GetParameterIndexByName()>: No such parameter
Warning in <NParameterList<unsigned long>::GetParameterIndexByName()>: No such parameter
Warning in <NParameterList<unsigned long>::GetParameterIndexByName()>: No such parameter
Warning in <NParameterList<unsigned long>::GetParameterIndexByName()>: No such parameter
Warning in <NParameterList<unsigned long>::GetParameterIndexByName()>: No such parameter
Warning in <NParameterList<unsigned long>::GetParameterIndexByName()>: No such parameter
Warning in <NParameterList<unsigned long>::GetParameterIndexByName()>: No such parameter
Warning in <NParamete

In [9]:
hRZ.Draw("colz")
c1.Draw()

In [10]:
hallchi.Draw()
c1.Draw()

In [11]:
hE.Draw()
c1.Draw()

In [12]:
hEchi.Draw("colz")
c1.Draw()

In [13]:
AoverEvschi.Draw("colz")
c1.Draw()